In [ ]:
import numpy as np
import pandas as pd

In [ ]:
import tensorflow as tf

In [ ]:
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input,decode_predictions
from tensorflow.keras.layers import Dense,Flatten
from tensorflow.keras.models import Model

In [ ]:
model=MobileNetV2(input_shape=(224,224,3),include_top=False,weights='imagenet')

In [ ]:
model.summary()

In [ ]:
from tensorflow.keras.layers import GlobalAveragePooling2D,Conv2D,AveragePooling2D
x=model.output
x=AveragePooling2D(pool_size=(7, 7))(x)
x=Conv2D(filters=3,kernel_size=(1,1))(x)
x=Flatten()(x)
predictions=Dense(1,activation='sigmoid')(x)

In [ ]:
new_model = Model(inputs=model.input,outputs=predictions)

In [ ]:
for layer in model.layers[:-5]:
    layer.trainable=False

In [ ]:
new_model.compile(optimizer='adam', loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
TRAIN_DIR = "training/"
HEIGHT = 224
WIDTH = 224
BATCH_SIZE = 25

train_datagen =  ImageDataGenerator(rescale=1./255,
      preprocessing_function=preprocess_input,
      rotation_range=90,
      horizontal_flip=True,
      vertical_flip=True
    )

train_generator = train_datagen.flow_from_directory(TRAIN_DIR, 
                                                    target_size=(HEIGHT, WIDTH), 
                                                    batch_size=BATCH_SIZE,class_mode='binary')
TEST_DIR = "testing/"
test_datagen =  ImageDataGenerator(rescale=1./255,preprocessing_function=preprocess_input)
validation_generator = test_datagen.flow_from_directory(TEST_DIR, 
                                                    target_size=(HEIGHT, WIDTH), 
                                                    class_mode='binary')

In [ ]:
new_model.fit_generator(train_generator,steps_per_epoch=106//BATCH_SIZE,epochs=31,validation_data=validation_generator)

In [ ]:
new_model.save('Demo.h5')

In [ ]:
crop_classifier=tf.keras.models.load_model('Demo.h5')

In [ ]:
crop_classifier.summary()